In [1]:
import pandas as pd
from sqlalchemy import create_engine

Get stats for every player in every regular season game for last five seasons:

In [2]:
engine = create_engine('sqlite:///D://Data//NFL.sqlite')
con = engine.connect()
player_stats = con.execute(
"""
SELECT 	sub2.*, team_abbreviation, position, weight, sub2.season - SUBSTR(birth_date,1,4) age
FROM
(
SELECT  sub1.*, sub1.player_id||'_'||season AS roster_id
FROM
(
SELECT 	Gameplay.*, 
		CASE WHEN CAST(SUBSTR(game_id,5,2) AS INT) < 4 THEN CAST(SUBSTR(game_id,1,4) AS INT) - 1
			ELSE CAST(SUBSTR(game_id,1,4) AS INT) END season,
		team_1, team_2
FROM Gameplay
LEFT JOIN 
(SELECT 	boxscore_index game_id,
		MAX(CASE WHEN r = 1 THEN opponent_abbr
			ELSE NULL END) AS team_1,
		MAX(CASE WHEN r = 2 THEN opponent_abbr
			ELSE NULL END) AS team_2
FROM
(
	SELECT boxscore_index, opponent_abbr, ROW_NUMBER() OVER(PARTITION BY boxscore_index) r
	FROM Games
	WHERE type='Reg'
	ORDER BY boxscore_index
)
GROUP BY game_id)
USING (game_id)
) AS sub1
) AS sub2
LEFT JOIN Rosters USING(roster_id)
WHERE sub2.season > 2015
"""
)
player_stats = pd.DataFrame(player_stats.fetchall(), columns=[
       'index', 'player_id', 'completed_passes', 'attempted_passes',
       'passing_yards', 'passing_touchdowns', 'interceptions_thrown',
       'times_sacked', 'yards_lost_from_sacks', 'longest_pass',
       'quarterback_rating', 'rush_attempts', 'rush_yards', 'rush_touchdowns',
       'longest_rush', 'times_pass_target', 'receptions', 'receiving_yards',
       'receiving_touchdowns', 'longest_reception', 'fumbles', 'fumbles_lost',
       'interceptions', 'yards_returned_from_interception',
       'interceptions_returned_for_touchdown', 'longest_interception_return',
       'passes_defended', 'sacks', 'combined_tackles', 'solo_tackles',
       'assists_on_tackles', 'tackles_for_loss', 'quarterback_hits',
       'fumbles_recovered', 'yards_recovered_from_fumble',
       'fumbles_recovered_for_touchdown', 'fumbles_forced', 'kickoff_returns',
       'kickoff_return_yards', 'average_kickoff_return_yards',
       'kickoff_return_touchdown', 'longest_kickoff_return', 'punt_returns',
       'punt_return_yards', 'yards_per_punt_return', 'punt_return_touchdown',
       'longest_punt_return', 'extra_points_made', 'extra_points_attempted',
       'field_goals_made', 'field_goals_attempted', 'punts',
       'total_punt_yards', 'yards_per_punt', 'longest_punt', 'game_id', 'season',
       'team_1', 'team_2', 'roster_id', 'team_abbreviation', 'position',
       'weight', 'age'
]).drop_duplicates()
con.close()

Fix team abbreviations for players that played on more than one team in a season or whose team_abbreviation is missing in API-supplied data:

In [3]:
player_stats = player_stats[(player_stats.team_1.isnull()==False)]
# The API mistakenly duplicates the fumble recovery yards column as fumbles recovered for touchdown
player_stats.drop(columns=['index', 'roster_id', 'fumbles_recovered_for_touchdown'], inplace=True)  
player_stats.reset_index(drop=True, inplace=True)

In [4]:
player_stats['pairs'] = list(zip(player_stats.team_1, player_stats.team_2))

In [5]:
player_stats.head()

,player_id,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,...,longest_punt,game_id,season,team_1,team_2,team_abbreviation,position,weight,age,pairs
0,NewtCa00,18.0,33.0,194.0,1.0,1.0,3.0,18.0,18.0,69.5,...,NaN,201609080den,2016,DEN,CAR,CAR,QB,245.0,27.0,"(DEN, CAR)"
1,StewJo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,201609080den,2016,DEN,CAR,CAR,RB,240.0,29.0,"(DEN, CAR)"
2,GinnTe00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,201609080den,2016,DEN,CAR,CAR,WR,180.0,31.0,"(DEN, CAR)"
3,WhitFo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,201609080den,2016,DEN,CAR,CAR,,205.0,27.0,"(DEN, CAR)"
4,TolbMi00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,201609080den,2016,DEN,CAR,CAR,,243.0,31.0,"(DEN, CAR)"


Standardize team abbreviations

In [6]:
print('Team_1 column:')
print(sorted(player_stats.team_1.unique()))
print('Team_abbreviation column:')
print(sorted(player_stats.team_abbreviation.fillna('').unique()))

Team_1 column:
['ATL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'CLT', 'CRD', 'DAL', 'DEN', 'DET', 'HTX', 'JAX', 'KAN', 'MIA', 'MIN', 'NOR', 'NWE', 'NYG', 'NYJ', 'OTI', 'PHI', 'PIT', 'RAI', 'RAM', 'RAV', 'SDG', 'SEA', 'SFO', 'TAM', 'WAS']
Team_abbreviation column:
['', '2TM', '3TM', 'ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GNB', 'HOU', 'IND', 'JAX', 'KAN', 'LAC', 'LAR', 'LVR', 'MIA', 'MIN', 'NOR', 'NWE', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SDG', 'SEA', 'SFO', 'TAM', 'TEN', 'WAS']


In [7]:
repl_dict = {'ARI':'CRD', 'BAL':'RAV', 'IND':'CLT', 'HOU':'HTX', 'LAC':'SDG', 'LAR':'RAM', 'LVR':'RAI',
            'OAK':'RAI', 'TEN':'OTI'}
for k,v in repl_dict.items():
    player_stats['team_abbreviation'] = player_stats.team_abbreviation.fillna('').str.replace(k,v)

In [8]:
print('Team_1 column:')
print(sorted(player_stats.team_1.unique()))
print('Team_2 column:')
print(sorted(player_stats.team_2.unique()))
print('Team_abbreviation column:')
print(sorted(player_stats.team_abbreviation.fillna('').unique()))

Team_1 column:
['ATL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'CLT', 'CRD', 'DAL', 'DEN', 'DET', 'HTX', 'JAX', 'KAN', 'MIA', 'MIN', 'NOR', 'NWE', 'NYG', 'NYJ', 'OTI', 'PHI', 'PIT', 'RAI', 'RAM', 'RAV', 'SDG', 'SEA', 'SFO', 'TAM', 'WAS']
Team_2 column:
['ATL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'CLT', 'CRD', 'DAL', 'DEN', 'DET', 'GNB', 'HTX', 'JAX', 'KAN', 'MIA', 'MIN', 'NOR', 'NWE', 'NYG', 'OTI', 'PHI', 'PIT', 'RAI', 'RAM', 'RAV', 'SDG', 'SEA', 'SFO', 'TAM', 'WAS']
Team_abbreviation column:
['', '2TM', '3TM', 'ATL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'CLT', 'CRD', 'DAL', 'DEN', 'DET', 'GNB', 'HTX', 'JAX', 'KAN', 'MIA', 'MIN', 'NOR', 'NWE', 'NYG', 'NYJ', 'OTI', 'PHI', 'PIT', 'RAI', 'RAM', 'RAV', 'SDG', 'SEA', 'SFO', 'TAM', 'WAS']


In [9]:
fixed = []
for i in player_stats.index:
    if player_stats.iloc[i,-5] in player_stats.iloc[i,-1]:
        fixed.append(player_stats.iloc[i,-5]) 
    else:
        fixed.append(None)

In [10]:
player_stats['team_abbr'] = fixed
player_stats.team_abbr.fillna(method='ffill', inplace=True)
player_stats.drop(columns=['team_abbreviation', 'team_1', 'team_2', 'pairs'], inplace=True)
player_stats.head()

,player_id,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,...,punts,total_punt_yards,yards_per_punt,longest_punt,game_id,season,position,weight,age,team_abbr
0,NewtCa00,18.0,33.0,194.0,1.0,1.0,3.0,18.0,18.0,69.5,...,NaN,NaN,NaN,NaN,201609080den,2016,QB,245.0,27.0,CAR
1,StewJo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,201609080den,2016,RB,240.0,29.0,CAR
2,GinnTe00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,201609080den,2016,WR,180.0,31.0,CAR
3,WhitFo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,201609080den,2016,,205.0,27.0,CAR
4,TolbMi00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,201609080den,2016,,243.0,31.0,CAR


Clean position column

In [11]:
player_stats['position'] = player_stats.position.str.upper()

In [12]:
player_stats.position.fillna('',inplace=True)

In [13]:
player_stats.loc[player_stats.position.str.startswith('/')==True,'position'] = \
player_stats.loc[player_stats.position.str.startswith('/')==True,'position'].str.replace('/','')

In [14]:
player_stats['position'] = [x.split('/')[0] if '/' in x else x for x in player_stats.position]

In [15]:
print(sorted(player_stats.position.unique()))

['', 'B', 'C', 'CB', 'DB', 'DE', 'DT', 'FB', 'FS', 'G', 'IL', 'ILB', 'K', 'LB', 'LCB', 'LDE', 'LDT', 'LG', 'LILB', 'LLB', 'LOLB', 'LT', 'ML', 'MLB', 'NT', 'OLB', 'P', 'QB', 'RB', 'RCB', 'RDE', 'RDT', 'RG', 'RILB', 'RLB', 'ROLB', 'RT', 'S', 'SS', 'T', 'TE', 'WR']


_Address weird position labels: B, IL, ML_

In [16]:
player_stats[player_stats.position=='B']

,player_id,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,...,punts,total_punt_yards,yards_per_punt,longest_punt,game_id,season,position,weight,age,team_abbr
31492,MixoJo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,201809090clt,2018,B,220.0,22.0,CIN
32326,MixoJo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,201809130cin,2018,B,220.0,22.0,CIN
35337,MixoJo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,201810070cin,2018,B,220.0,22.0,CIN
36180,MixoJo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,201810140cin,2018,B,220.0,22.0,CIN
37180,MixoJo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,201810210kan,2018,B,220.0,22.0,CIN
38006,MixoJo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,201810280cin,2018,B,220.0,22.0,CIN
39587,MixoJo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,201811110cin,2018,B,220.0,22.0,CIN
40834,MixoJo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,201811180rav,2018,B,220.0,22.0,CIN
41407,MixoJo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,201811250cin,2018,B,220.0,22.0,CIN
42136,MixoJo00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,201812020cin,2018,B,220.0,22.0,CIN


In [17]:
player_stats['position'] = ['RB' if x == 'B' else x for x in player_stats.position]

In [18]:
player_stats[player_stats.position=='ML'].dropna(axis=1,how='all')

,player_id,interceptions,yards_returned_from_interception,interceptions_returned_for_touchdown,longest_interception_return,passes_defended,sacks,combined_tackles,solo_tackles,assists_on_tackles,...,quarterback_hits,fumbles_recovered,yards_recovered_from_fumble,fumbles_forced,game_id,season,position,weight,age,team_abbr
46938,SchoJo00,0.0,0.0,0.0,0.0,0.0,0.0,8.0,5.0,3.0,...,0.0,0.0,0.0,0.0,201909080cle,2019,ML,245.0,26.0,CLE
48555,SchoJo00,0.0,0.0,0.0,0.0,0.0,0.0,13.0,10.0,3.0,...,0.0,0.0,0.0,1.0,201909160nyj,2019,ML,245.0,26.0,CLE
48747,SchoJo00,0.0,0.0,0.0,0.0,1.0,0.0,6.0,3.0,3.0,...,0.0,1.0,21.0,0.0,201909220cle,2019,ML,245.0,26.0,CLE
50240,SchoJo00,0.0,0.0,0.0,0.0,0.0,1.0,16.0,9.0,7.0,...,1.0,0.0,0.0,0.0,201909290rav,2019,ML,245.0,26.0,CLE
51130,SchoJo00,0.0,0.0,0.0,0.0,0.0,0.0,11.0,11.0,0.0,...,0.0,0.0,0.0,0.0,201910070sfo,2019,ML,245.0,26.0,CLE
51269,SchoJo00,0.0,0.0,0.0,0.0,0.0,0.0,8.0,5.0,3.0,...,1.0,0.0,0.0,0.0,201910130cle,2019,ML,245.0,26.0,CLE
53332,SchoJo00,0.0,0.0,0.0,0.0,0.0,0.0,11.0,8.0,3.0,...,0.0,0.0,0.0,0.0,201910270nwe,2019,ML,245.0,26.0,CLE
53790,SchoJo00,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,...,0.0,0.0,0.0,1.0,201911030den,2019,ML,245.0,26.0,CLE
54610,SchoJo00,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,...,0.0,0.0,0.0,0.0,201911100cle,2019,ML,245.0,26.0,CLE
55177,SchoJo00,2.0,20.0,0.0,20.0,4.0,1.0,10.0,7.0,3.0,...,2.0,0.0,0.0,0.0,201911140cle,2019,ML,245.0,26.0,CLE


In [19]:
player_stats['position'] = ['MLB' if x == 'ML' else x for x in player_stats.position]

In [20]:
player_stats[player_stats.position=='IL'].dropna(axis=1,how='all').describe().T

,count,mean,std,min,25%,50%,75%,max
completed_passes,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
attempted_passes,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
passing_yards,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
passing_touchdowns,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
interceptions_thrown,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
times_sacked,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
yards_lost_from_sacks,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
longest_pass,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
rush_attempts,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
rush_yards,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0


In [21]:
player_stats['position'] = ['ILB' if x == 'IL' else x for x in player_stats.position]

Create position group labels from existing position labels

In [22]:
position_group = []
for position in player_stats.position:
    if position in ['FB', 'RB']:
        position_group.append('RunningBacks')
    elif position in ['C', 'G', 'T', 'LG', 'RG', 'LT', 'RT']:
        position_group.append('O_Line')
    elif position == 'QB':
        position_group.append('Quarterbacks')
    elif position in ['TE', 'WR']:
        position_group.append('Receivers')
    elif position in ['CB', 'DB', 'FS', 'SS', 'S', 'LCB', 'RCB']:
        position_group.append('Secondary')
    elif 'LB' in position:
        position_group.append('Linebackers')
    elif position in ['K','P']:
        position_group.append('Kickers')
    elif position == '':
        position_group.append('Other')
    else:
        position_group.append('D_Line')
pd.Series(position_group).value_counts()

Other           28596
Secondary       12297
Receivers       10984
D_Line           9289
Linebackers      8717
RunningBacks     3699
Quarterbacks     2541
Kickers           902
O_Line            167
dtype: int64

In [23]:
player_stats['position_group'] = position_group

Assign a position group label for players without a position label

_If player has no position label, had at least one pass thrown to them, is not a secondary player and rushed the ball fewer than three times, then assign to Receivers_

In [24]:
player_stats.loc[(player_stats.position_group=='Other')&(player_stats.times_pass_target>0)\
                &((player_stats.passes_defended.isnull()==True)|(player_stats.passes_defended==0))\
                &(player_stats.rush_attempts<3),'position_group'] = 'Receivers'

_If player has no position label and has punted or attempted a field goal or extra point, then assign to Kickers_

In [25]:
player_stats.loc[(player_stats.position_group=='Other')&((player_stats.punts>=1)|\
                (player_stats.extra_points_attempted>=1)|\
                (player_stats.field_goals_attempted>=1)),'position_group'] = 'Kickers'

_If player has no position labeland has attempted more than one pass, then assign to Quarterbacks_

In [26]:
player_stats.loc[(player_stats.position_group=='Other')&(player_stats.attempted_passes>1),
                 'position_group'] = 'Quarterbacks'

_If player has no position label and has attempted at least three rushes, then assign to RunningBacks_

In [27]:
player_stats.loc[(player_stats.position_group=='Other')&(player_stats.rush_attempts>=3),
                 'position_group'] = 'RunningBacks'

_If player has no position label and has returned a punt or kickoff, then assign to Returners_

In [28]:
player_stats.loc[(player_stats.position_group=='Other')\
                 &(player_stats.punt_returns + player_stats.kickoff_returns >0),
                 'position_group'] = 'Returners'

_If player has no position label and weighs at least 270 pounds, then assign to D_Line_

In [29]:
player_stats.loc[(player_stats.position_group=='Other')&(player_stats.weight>=270),
                'position_group'] = 'D_Line'

_If player has no position label and weighs at least 225 pounds, then assign to Linebackers_

In [30]:
player_stats.loc[(player_stats.position_group=='Other')&(player_stats.weight>=225),
                 'position_group'] = 'Linebackers'

_If player still has no position label, then assign to Secondary_

In [31]:
player_stats.loc[(player_stats.position_group=='Other'),
                 'position_group'] = 'Secondary'

In [32]:
player_stats.position_group.value_counts()

Secondary       18255
Receivers       16272
Linebackers     14320
D_Line          13089
RunningBacks     5718
Kickers          5058
Quarterbacks     2833
Returners        1480
O_Line            167
Name: position_group, dtype: int64

Since offensive linemen are only in this dataset if they made a tackle after a turnover, drop them from the dataset. Then group data by game, team and position group.

In [33]:
gameplay = player_stats[player_stats.position_group!='O_Line'].drop(columns=[
    'season','weight','age']).groupby(by=['game_id', 'team_abbr', 'position_group'],
                                 as_index=False).agg('sum')

Get dataframes of each position group.

In [34]:
def get_pg(df, pg, value_list):
    df2 = df[df['position_group']==pg].pivot(index=['game_id', 'team_abbr'], columns='position_group',
                              values=value_list).reset_index()
    df2.columns = ['_'.join(col) for col in df2.columns.values]
    return df2

In [35]:
dline_values = ['interceptions', 'yards_returned_from_interception',
               'interceptions_returned_for_touchdown', 'longest_interception_return',
               'passes_defended', 'sacks', 'combined_tackles', 'solo_tackles',
               'assists_on_tackles', 'tackles_for_loss', 'quarterback_hits',
               'fumbles_recovered', 'yards_recovered_from_fumble',
               'fumbles_forced']

dline = get_pg(gameplay, 'D_Line', dline_values)
dline   

,game_id_,team_abbr_,interceptions_D_Line,yards_returned_from_interception_D_Line,interceptions_returned_for_touchdown_D_Line,longest_interception_return_D_Line,passes_defended_D_Line,sacks_D_Line,combined_tackles_D_Line,solo_tackles_D_Line,assists_on_tackles_D_Line,tackles_for_loss_D_Line,quarterback_hits_D_Line,fumbles_recovered_D_Line,yards_recovered_from_fumble_D_Line,fumbles_forced_D_Line
0,201609080den,CAR,0.0,0.0,0.0,0.0,2.0,1.0,5.0,5.0,0.0,1.0,1.0,0.0,0.0,1.0
1,201609080den,DEN,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0
2,201609110atl,ATL,0.0,0.0,0.0,0.0,0.0,0.0,11.0,9.0,2.0,1.0,2.0,0.0,0.0,0.0
3,201609110atl,TAM,0.0,0.0,0.0,0.0,2.0,2.0,6.0,6.0,0.0,3.0,2.0,0.0,0.0,0.0
4,201609110clt,CLT,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0,5.0,1.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,202101030ram,RAM,0.0,0.0,0.0,0.0,0.0,1.0,10.0,7.0,3.0,4.0,2.0,0.0,0.0,0.0
2556,202101030sfo,SEA,0.0,0.0,0.0,0.0,1.0,1.0,6.0,5.0,1.0,1.0,6.0,1.0,0.0,0.0
2557,202101030sfo,SFO,0.0,0.0,0.0,0.0,1.0,0.0,11.0,4.0,7.0,0.0,1.0,0.0,0.0,0.0
2558,202101030tam,ATL,0.0,0.0,0.0,0.0,0.0,1.0,12.0,6.0,6.0,1.0,1.0,0.0,0.0,0.0


In [36]:
dline.rename(columns={'game_id_':'game_id', 'team_abbr_':'team_abbr'}, inplace=True)
dline['turnovers_recovered_D_Line'] = dline.interceptions_D_Line + dline.fumbles_recovered_D_Line
dline['turnover_yards_D_Line'] = dline.yards_returned_from_interception_D_Line + dline.yards_recovered_from_fumble_D_Line

dline.drop(columns=['interceptions_D_Line', 'fumbles_recovered_D_Line', 
                   'yards_returned_from_interception_D_Line', 'yards_recovered_from_fumble_D_Line',
                   'longest_interception_return_D_Line', 'solo_tackles_D_Line', 'assists_on_tackles_D_Line'],
          inplace=True)
dline

,game_id,team_abbr,interceptions_returned_for_touchdown_D_Line,passes_defended_D_Line,sacks_D_Line,combined_tackles_D_Line,tackles_for_loss_D_Line,quarterback_hits_D_Line,fumbles_forced_D_Line,turnovers_recovered_D_Line,turnover_yards_D_Line
0,201609080den,CAR,0.0,2.0,1.0,5.0,1.0,1.0,1.0,0.0,0.0
1,201609080den,DEN,0.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,0.0
2,201609110atl,ATL,0.0,0.0,0.0,11.0,1.0,2.0,0.0,0.0,0.0
3,201609110atl,TAM,0.0,2.0,2.0,6.0,3.0,2.0,0.0,0.0,0.0
4,201609110clt,CLT,0.0,0.0,0.0,7.0,1.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2555,202101030ram,RAM,0.0,0.0,1.0,10.0,4.0,2.0,0.0,0.0,0.0
2556,202101030sfo,SEA,0.0,1.0,1.0,6.0,1.0,6.0,0.0,1.0,0.0
2557,202101030sfo,SFO,0.0,1.0,0.0,11.0,0.0,1.0,0.0,0.0,0.0
2558,202101030tam,ATL,0.0,0.0,1.0,12.0,1.0,1.0,0.0,0.0,0.0


In [37]:
lb_values = ['interceptions', 'yards_returned_from_interception', 'interceptions_returned_for_touchdown',
           'passes_defended', 'sacks', 'combined_tackles', 'solo_tackles',
           'assists_on_tackles', 'tackles_for_loss', 'quarterback_hits',
           'fumbles_recovered', 'yards_recovered_from_fumble',
           'fumbles_forced']

lb = get_pg(gameplay, 'Linebackers', lb_values)
lb

,game_id_,team_abbr_,interceptions_Linebackers,yards_returned_from_interception_Linebackers,interceptions_returned_for_touchdown_Linebackers,passes_defended_Linebackers,sacks_Linebackers,combined_tackles_Linebackers,solo_tackles_Linebackers,assists_on_tackles_Linebackers,tackles_for_loss_Linebackers,quarterback_hits_Linebackers,fumbles_recovered_Linebackers,yards_recovered_from_fumble_Linebackers,fumbles_forced_Linebackers
0,201609080den,CAR,1.0,4.0,0.0,1.0,0.0,20.0,20.0,0.0,3.0,0.0,1.0,0.0,0.0
1,201609080den,DEN,0.0,0.0,0.0,1.0,3.0,24.0,21.0,3.0,2.0,6.0,0.0,0.0,0.0
2,201609110atl,ATL,0.0,0.0,0.0,1.0,0.0,15.0,13.0,2.0,1.0,4.0,0.0,0.0,0.0
3,201609110atl,TAM,0.0,0.0,0.0,0.0,1.0,29.0,25.0,4.0,5.0,2.0,0.0,0.0,0.0
4,201609110clt,CLT,0.0,0.0,0.0,0.0,1.0,25.0,18.0,7.0,2.0,3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,202101030ram,RAM,0.0,0.0,0.0,0.0,3.0,19.0,15.0,4.0,4.0,3.0,0.0,0.0,1.0
2556,202101030sfo,SEA,0.0,0.0,0.0,0.0,2.0,26.0,18.0,8.0,2.0,1.0,0.0,0.0,1.0
2557,202101030sfo,SFO,0.0,0.0,0.0,1.0,1.0,20.0,15.0,5.0,2.0,2.0,0.0,0.0,0.0
2558,202101030tam,ATL,0.0,0.0,0.0,0.0,0.0,15.0,8.0,7.0,0.0,1.0,0.0,0.0,0.0


In [38]:
lb.rename(columns={'game_id_':'game_id', 'team_abbr_':'team_abbr'}, inplace=True)
lb['turnovers_recovered_Linebackers'] = lb.interceptions_Linebackers + lb.fumbles_recovered_Linebackers
lb['turnover_yards_Linebackers'] = lb.yards_returned_from_interception_Linebackers + lb.yards_recovered_from_fumble_Linebackers
lb.drop(columns=['interceptions_Linebackers', 'fumbles_recovered_Linebackers', 
                   'yards_returned_from_interception_Linebackers', 'yards_recovered_from_fumble_Linebackers',
                   'solo_tackles_Linebackers', 'assists_on_tackles_Linebackers'],
          inplace=True)
lb

,game_id,team_abbr,interceptions_returned_for_touchdown_Linebackers,passes_defended_Linebackers,sacks_Linebackers,combined_tackles_Linebackers,tackles_for_loss_Linebackers,quarterback_hits_Linebackers,fumbles_forced_Linebackers,turnovers_recovered_Linebackers,turnover_yards_Linebackers
0,201609080den,CAR,0.0,1.0,0.0,20.0,3.0,0.0,0.0,2.0,4.0
1,201609080den,DEN,0.0,1.0,3.0,24.0,2.0,6.0,0.0,0.0,0.0
2,201609110atl,ATL,0.0,1.0,0.0,15.0,1.0,4.0,0.0,0.0,0.0
3,201609110atl,TAM,0.0,0.0,1.0,29.0,5.0,2.0,0.0,0.0,0.0
4,201609110clt,CLT,0.0,0.0,1.0,25.0,2.0,3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2555,202101030ram,RAM,0.0,0.0,3.0,19.0,4.0,3.0,1.0,0.0,0.0
2556,202101030sfo,SEA,0.0,0.0,2.0,26.0,2.0,1.0,1.0,0.0,0.0
2557,202101030sfo,SFO,0.0,1.0,1.0,20.0,2.0,2.0,0.0,0.0,0.0
2558,202101030tam,ATL,0.0,0.0,0.0,15.0,0.0,1.0,0.0,0.0,0.0


In [39]:
qb_values = ['completed_passes', 'attempted_passes', 'passing_yards', 'passing_touchdowns',
           'interceptions_thrown', 'times_sacked', 'yards_lost_from_sacks','rush_attempts', 
           'rush_yards', 'rush_touchdowns', 'fumbles', 'fumbles_lost', 'quarterback_rating']

qb = get_pg(gameplay, 'Quarterbacks', qb_values)
qb

,game_id_,team_abbr_,completed_passes_Quarterbacks,attempted_passes_Quarterbacks,passing_yards_Quarterbacks,passing_touchdowns_Quarterbacks,interceptions_thrown_Quarterbacks,times_sacked_Quarterbacks,yards_lost_from_sacks_Quarterbacks,rush_attempts_Quarterbacks,rush_yards_Quarterbacks,rush_touchdowns_Quarterbacks,fumbles_Quarterbacks,fumbles_lost_Quarterbacks,quarterback_rating_Quarterbacks
0,201609080den,CAR,18.0,33.0,194.0,1.0,1.0,3.0,18.0,11.0,54.0,1.0,0.0,0.0,69.5
1,201609080den,DEN,18.0,26.0,178.0,1.0,2.0,2.0,19.0,5.0,20.0,0.0,0.0,0.0,69.1
2,201609110atl,ATL,27.0,39.0,334.0,2.0,0.0,3.0,12.0,3.0,10.0,0.0,0.0,0.0,112.6
3,201609110atl,TAM,23.0,32.0,281.0,4.0,1.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,125.1
4,201609110clt,CLT,31.0,47.0,385.0,4.0,0.0,2.0,17.0,3.0,21.0,0.0,0.0,0.0,119.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,202101030ram,RAM,22.0,38.0,231.0,0.0,1.0,2.0,8.0,6.0,56.0,0.0,0.0,0.0,64.7
2550,202101030sfo,SEA,20.0,36.0,181.0,2.0,0.0,2.0,22.0,5.0,29.0,0.0,0.0,0.0,87.8
2551,202101030sfo,SFO,25.0,37.0,273.0,1.0,0.0,3.0,31.0,3.0,10.0,0.0,1.0,1.0,98.1
2552,202101030tam,ATL,29.0,44.0,265.0,2.0,0.0,1.0,7.0,3.0,1.0,1.0,2.0,1.0,97.3


In [40]:
qb.rename(columns={'game_id_':'game_id', 'team_abbr_':'team_abbr'}, inplace=True)
qb['compl_perc_Quarterbacks'] = round(qb.completed_passes_Quarterbacks/qb.attempted_passes_Quarterbacks,2)
qb

,game_id,team_abbr,completed_passes_Quarterbacks,attempted_passes_Quarterbacks,passing_yards_Quarterbacks,passing_touchdowns_Quarterbacks,interceptions_thrown_Quarterbacks,times_sacked_Quarterbacks,yards_lost_from_sacks_Quarterbacks,rush_attempts_Quarterbacks,rush_yards_Quarterbacks,rush_touchdowns_Quarterbacks,fumbles_Quarterbacks,fumbles_lost_Quarterbacks,quarterback_rating_Quarterbacks,compl_perc_Quarterbacks
0,201609080den,CAR,18.0,33.0,194.0,1.0,1.0,3.0,18.0,11.0,54.0,1.0,0.0,0.0,69.5,0.55
1,201609080den,DEN,18.0,26.0,178.0,1.0,2.0,2.0,19.0,5.0,20.0,0.0,0.0,0.0,69.1,0.69
2,201609110atl,ATL,27.0,39.0,334.0,2.0,0.0,3.0,12.0,3.0,10.0,0.0,0.0,0.0,112.6,0.69
3,201609110atl,TAM,23.0,32.0,281.0,4.0,1.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,125.1,0.72
4,201609110clt,CLT,31.0,47.0,385.0,4.0,0.0,2.0,17.0,3.0,21.0,0.0,0.0,0.0,119.5,0.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,202101030ram,RAM,22.0,38.0,231.0,0.0,1.0,2.0,8.0,6.0,56.0,0.0,0.0,0.0,64.7,0.58
2550,202101030sfo,SEA,20.0,36.0,181.0,2.0,0.0,2.0,22.0,5.0,29.0,0.0,0.0,0.0,87.8,0.56
2551,202101030sfo,SFO,25.0,37.0,273.0,1.0,0.0,3.0,31.0,3.0,10.0,0.0,1.0,1.0,98.1,0.68
2552,202101030tam,ATL,29.0,44.0,265.0,2.0,0.0,1.0,7.0,3.0,1.0,1.0,2.0,1.0,97.3,0.66


In [41]:
rb_values = ['rush_attempts', 'rush_yards', 'rush_touchdowns',
            'times_pass_target', 'receptions', 'receiving_yards', 'receiving_touchdowns',
            'fumbles', 'fumbles_lost']

rb = get_pg(gameplay, 'RunningBacks', rb_values)
rb.rename(columns={'game_id_':'game_id', 'team_abbr_':'team_abbr'}, inplace=True)
col_no_spaces = rb.columns.str.replace(' ','_')
rb.rename(columns=dict(zip(rb.columns, col_no_spaces)),inplace=True)
rb['target_catch_perc_RunningBacks'] = round(rb.receptions_RunningBacks/rb.times_pass_target_RunningBacks,2)
rb

,game_id,team_abbr,rush_attempts_RunningBacks,rush_yards_RunningBacks,rush_touchdowns_RunningBacks,times_pass_target_RunningBacks,receptions_RunningBacks,receiving_yards_RunningBacks,receiving_touchdowns_RunningBacks,fumbles_RunningBacks,fumbles_lost_RunningBacks,target_catch_perc_RunningBacks
0,201609080den,CAR,18.0,77.0,0.0,2.0,1.0,4.0,0.0,0.0,0.0,0.50
1,201609080den,DEN,24.0,128.0,2.0,5.0,4.0,47.0,1.0,1.0,1.0,0.80
2,201609110atl,ATL,19.0,42.0,0.0,11.0,10.0,123.0,0.0,0.0,0.0,0.91
3,201609110atl,TAM,22.0,71.0,0.0,8.0,8.0,66.0,1.0,0.0,0.0,1.00
4,201609110clt,CLT,14.0,59.0,0.0,6.0,4.0,19.0,0.0,0.0,0.0,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...
2555,202101030ram,RAM,24.0,54.0,0.0,5.0,5.0,50.0,0.0,1.0,1.0,1.00
2556,202101030sfo,SEA,22.0,92.0,1.0,2.0,2.0,39.0,0.0,0.0,0.0,1.00
2557,202101030sfo,SFO,21.0,76.0,1.0,10.0,5.0,22.0,1.0,0.0,0.0,0.50
2558,202101030tam,ATL,21.0,121.0,0.0,9.0,5.0,66.0,0.0,1.0,0.0,0.56


In [42]:
secondary_values = ['interceptions', 'yards_returned_from_interception', 'interceptions_returned_for_touchdown',
                   'passes_defended', 'sacks', 'combined_tackles', 'solo_tackles',
                   'assists_on_tackles', 'tackles_for_loss', 'quarterback_hits',
                   'fumbles_recovered', 'yards_recovered_from_fumble',
                   'fumbles_forced']

secondary = get_pg(gameplay, 'Secondary', secondary_values)
secondary.rename(columns={'game_id_':'game_id', 'team_abbr_':'team_abbr'}, inplace=True)
secondary['turnovers_recovered_Secondary'] = secondary.interceptions_Secondary + secondary.fumbles_recovered_Secondary
secondary['turnover_yards_Secondary'] = secondary.yards_returned_from_interception_Secondary + secondary.yards_recovered_from_fumble_Secondary
secondary.drop(columns=['interceptions_Secondary', 'fumbles_recovered_Secondary', 
                   'yards_returned_from_interception_Secondary', 'yards_recovered_from_fumble_Secondary',
                   'solo_tackles_Secondary', 'assists_on_tackles_Secondary'],
          inplace=True)
secondary

,game_id,team_abbr,interceptions_returned_for_touchdown_Secondary,passes_defended_Secondary,sacks_Secondary,combined_tackles_Secondary,tackles_for_loss_Secondary,quarterback_hits_Secondary,fumbles_forced_Secondary,turnovers_recovered_Secondary,turnover_yards_Secondary
0,201609080den,CAR,0.0,3.0,1.0,19.0,1.0,2.0,0.0,1.0,1.0
1,201609080den,DEN,0.0,5.0,0.0,23.0,0.0,0.0,0.0,1.0,0.0
2,201609110atl,ATL,0.0,3.0,0.0,27.0,0.0,0.0,0.0,1.0,9.0
3,201609110atl,TAM,0.0,4.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0
4,201609110clt,CLT,0.0,3.0,0.0,29.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2555,202101030ram,RAM,1.0,6.0,0.0,20.0,0.0,0.0,0.0,1.0,84.0
2556,202101030sfo,SEA,0.0,1.0,0.0,28.0,2.0,0.0,0.0,0.0,0.0
2557,202101030sfo,SFO,0.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,0.0
2558,202101030tam,ATL,0.0,2.0,0.0,28.0,1.0,0.0,0.0,1.0,10.0


In [43]:
wr_values = ['times_pass_target', 'receptions', 'receiving_yards', 'receiving_touchdowns',
            'fumbles', 'fumbles_lost']
wr = get_pg(gameplay, 'Receivers', wr_values)
wr.rename(columns={'game_id_':'game_id', 'team_abbr_':'team_abbr'}, inplace=True)
wr['target_catch_perc_Receivers'] = round(wr.receptions_Receivers/wr.times_pass_target_Receivers,2)
wr

,game_id,team_abbr,times_pass_target_Receivers,receptions_Receivers,receiving_yards_Receivers,receiving_touchdowns_Receivers,fumbles_Receivers,fumbles_lost_Receivers,target_catch_perc_Receivers
0,201609080den,CAR,29.0,17.0,190.0,1.0,0.0,0.0,0.59
1,201609080den,DEN,21.0,14.0,131.0,0.0,0.0,0.0,0.67
2,201609110atl,ATL,27.0,17.0,211.0,2.0,0.0,0.0,0.63
3,201609110atl,TAM,24.0,15.0,215.0,3.0,0.0,0.0,0.62
4,201609110clt,CLT,39.0,27.0,366.0,4.0,0.0,0.0,0.69
...,...,...,...,...,...,...,...,...,...
2555,202101030ram,RAM,32.0,17.0,181.0,0.0,0.0,0.0,0.53
2556,202101030sfo,SEA,30.0,18.0,142.0,2.0,0.0,0.0,0.60
2557,202101030sfo,SFO,25.0,20.0,251.0,0.0,0.0,0.0,0.80
2558,202101030tam,ATL,32.0,24.0,199.0,2.0,1.0,1.0,0.75


In [44]:
st_values = ['extra_points_made', 'extra_points_attempted',
           'field_goals_made', 'field_goals_attempted', 'punts',
           'total_punt_yards', 'yards_per_punt']
st = get_pg(gameplay, 'Kickers', st_values)
st.rename(columns={'game_id_':'game_id', 'team_abbr_':'team_abbr'}, inplace=True)
col_no_spaces = st.columns.str.replace(' ','_')
st.rename(columns=dict(zip(st.columns, col_no_spaces)),inplace=True)
st

,game_id,team_abbr,extra_points_made_Kickers,extra_points_attempted_Kickers,field_goals_made_Kickers,field_goals_attempted_Kickers,punts_Kickers,total_punt_yards_Kickers,yards_per_punt_Kickers
0,201609080den,CAR,2.0,2.0,2.0,3.0,4.0,252.0,63.0
1,201609080den,DEN,3.0,3.0,0.0,0.0,3.0,140.0,46.7
2,201609110atl,ATL,1.0,1.0,3.0,3.0,5.0,193.0,38.6
3,201609110atl,TAM,4.0,4.0,1.0,1.0,5.0,247.0,49.4
4,201609110clt,CLT,0.0,0.0,0.0,0.0,4.0,195.0,48.8
...,...,...,...,...,...,...,...,...,...
2554,202101030ram,RAM,2.0,2.0,3.0,4.0,5.0,222.0,44.4
2555,202101030sfo,SEA,2.0,3.0,2.0,2.0,5.0,266.0,53.2
2556,202101030sfo,SFO,2.0,2.0,3.0,3.0,5.0,227.0,45.4
2557,202101030tam,ATL,3.0,3.0,2.0,2.0,1.0,43.0,43.0


_Since some returners would have already been grouped into receiver or running back groups, I'll sum the full data set rather than the Returners position group_

In [45]:
returns = player_stats[['game_id','team_abbr', 'kickoff_returns', 'kickoff_return_yards',
            'kickoff_return_touchdown', 'punt_returns',
            'punt_return_yards', 'punt_return_touchdown']].groupby(by=['game_id','team_abbr'],
            as_index=False).agg('sum')
returns

,game_id,team_abbr,kickoff_returns,kickoff_return_yards,kickoff_return_touchdown,punt_returns,punt_return_yards,punt_return_touchdown
0,201609080den,CAR,1.0,9.0,0.0,2.0,12.0,0.0
1,201609080den,DEN,0.0,0.0,0.0,2.0,26.0,0.0
2,201609110atl,ATL,3.0,85.0,0.0,2.0,22.0,0.0
3,201609110atl,TAM,1.0,12.0,0.0,0.0,0.0,0.0
4,201609110clt,CLT,2.0,25.0,0.0,2.0,13.0,0.0
...,...,...,...,...,...,...,...,...
2559,202101030ram,RAM,3.0,63.0,0.0,0.0,0.0,0.0
2560,202101030sfo,SEA,0.0,0.0,0.0,2.0,14.0,0.0
2561,202101030sfo,SFO,2.0,29.0,0.0,4.0,36.0,0.0
2562,202101030tam,ATL,2.0,53.0,0.0,0.0,0.0,0.0


Combine position group dataframes

In [46]:
gameplay = player_stats[['game_id', 'team_abbr']].drop_duplicates().reset_index(drop=True)

gameplay = gameplay.merge(qb, on=['game_id','team_abbr'], how='left')
gameplay = gameplay.merge(rb, on=['game_id','team_abbr'], how='left')
gameplay = gameplay.merge(wr, on=['game_id','team_abbr'], how='left')
gameplay = gameplay.merge(dline, on=['game_id','team_abbr'], how='left')
gameplay = gameplay.merge(lb, on=['game_id','team_abbr'], how='left')
gameplay = gameplay.merge(secondary, on=['game_id','team_abbr'], how='left')
gameplay = gameplay.merge(st, on=['game_id','team_abbr'], how='left') 
gameplay = gameplay.merge(returns, on=['game_id','team_abbr'], how='left') 

gameplay.head()

,game_id,team_abbr,completed_passes_Quarterbacks,attempted_passes_Quarterbacks,passing_yards_Quarterbacks,passing_touchdowns_Quarterbacks,interceptions_thrown_Quarterbacks,times_sacked_Quarterbacks,yards_lost_from_sacks_Quarterbacks,rush_attempts_Quarterbacks,...,field_goals_attempted_Kickers,punts_Kickers,total_punt_yards_Kickers,yards_per_punt_Kickers,kickoff_returns,kickoff_return_yards,kickoff_return_touchdown,punt_returns,punt_return_yards,punt_return_touchdown
0,201609080den,CAR,18.0,33.0,194.0,1.0,1.0,3.0,18.0,11.0,...,3.0,4.0,252.0,63.0,1.0,9.0,0.0,2.0,12.0,0.0
1,201609080den,DEN,18.0,26.0,178.0,1.0,2.0,2.0,19.0,5.0,...,0.0,3.0,140.0,46.7,0.0,0.0,0.0,2.0,26.0,0.0
2,201609110atl,TAM,23.0,32.0,281.0,4.0,1.0,0.0,0.0,4.0,...,1.0,5.0,247.0,49.4,1.0,12.0,0.0,0.0,0.0,0.0
3,201609110atl,ATL,27.0,39.0,334.0,2.0,0.0,3.0,12.0,3.0,...,3.0,5.0,193.0,38.6,3.0,85.0,0.0,2.0,22.0,0.0
4,201609110clt,DET,31.0,39.0,340.0,3.0,0.0,1.0,8.0,2.0,...,3.0,4.0,235.0,58.8,1.0,22.0,0.0,1.0,-1.0,0.0


In [47]:
gameplay.shape

(2564, 73)

Get win/loss data

In [48]:
engine = create_engine('sqlite:///D://Data//NFL.sqlite')
con = engine.connect()
wins = con.execute(
"""
SELECT boxscore_index game_id, result, opponent_abbr
FROM Games
WHERE season > 2015 AND type = 'Reg'
"""
)
wins = pd.DataFrame(wins.fetchall(), columns=['game_id', 'result', 'opponent_abbr'])
con.close()
wins

,game_id,result,opponent_abbr
0,202009130min,Win,MIN
1,202009200gnb,Win,DET
2,202009270nor,Win,NOR
3,202010050gnb,Win,ATL
4,202010180tam,Loss,TAM
...,...,...,...
2555,201612050nyj,Loss,CLT
2556,201612110sfo,Win,SFO
2557,201612170nyj,Loss,MIA
2558,201612240nwe,Loss,NWE


In [49]:
wins['result'] = ['Loss' if x == 'Win' else 'Win' for x in wins.result]
wins.rename(columns={'opponent_abbr':'team_abbr'}, inplace=True)
wins

,game_id,result,team_abbr
0,202009130min,Loss,MIN
1,202009200gnb,Loss,DET
2,202009270nor,Loss,NOR
3,202010050gnb,Loss,ATL
4,202010180tam,Win,TAM
...,...,...,...
2555,201612050nyj,Win,CLT
2556,201612110sfo,Loss,SFO
2557,201612170nyj,Win,MIA
2558,201612240nwe,Win,NWE


Combine win/loss data with gameplay data 

In [50]:
gameplay = gameplay.merge(wins, on=['game_id','team_abbr'], how='left') 
gameplay

,game_id,team_abbr,completed_passes_Quarterbacks,attempted_passes_Quarterbacks,passing_yards_Quarterbacks,passing_touchdowns_Quarterbacks,interceptions_thrown_Quarterbacks,times_sacked_Quarterbacks,yards_lost_from_sacks_Quarterbacks,rush_attempts_Quarterbacks,...,punts_Kickers,total_punt_yards_Kickers,yards_per_punt_Kickers,kickoff_returns,kickoff_return_yards,kickoff_return_touchdown,punt_returns,punt_return_yards,punt_return_touchdown,result
0,201609080den,CAR,18.0,33.0,194.0,1.0,1.0,3.0,18.0,11.0,...,4.0,252.0,63.0,1.0,9.0,0.0,2.0,12.0,0.0,Loss
1,201609080den,DEN,18.0,26.0,178.0,1.0,2.0,2.0,19.0,5.0,...,3.0,140.0,46.7,0.0,0.0,0.0,2.0,26.0,0.0,Win
2,201609110atl,TAM,23.0,32.0,281.0,4.0,1.0,0.0,0.0,4.0,...,5.0,247.0,49.4,1.0,12.0,0.0,0.0,0.0,0.0,Win
3,201609110atl,ATL,27.0,39.0,334.0,2.0,0.0,3.0,12.0,3.0,...,5.0,193.0,38.6,3.0,85.0,0.0,2.0,22.0,0.0,Loss
4,201609110clt,DET,31.0,39.0,340.0,3.0,0.0,1.0,8.0,2.0,...,4.0,235.0,58.8,1.0,22.0,0.0,1.0,-1.0,0.0,Win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2559,202101030ram,RAM,22.0,38.0,231.0,0.0,1.0,2.0,8.0,6.0,...,5.0,222.0,44.4,3.0,63.0,0.0,0.0,0.0,0.0,Win
2560,202101030sfo,SEA,20.0,36.0,181.0,2.0,0.0,2.0,22.0,5.0,...,5.0,266.0,53.2,0.0,0.0,0.0,2.0,14.0,0.0,Win
2561,202101030sfo,SFO,25.0,37.0,273.0,1.0,0.0,3.0,31.0,3.0,...,5.0,227.0,45.4,2.0,29.0,0.0,4.0,36.0,0.0,Loss
2562,202101030tam,ATL,29.0,44.0,265.0,2.0,0.0,1.0,7.0,3.0,...,1.0,43.0,43.0,2.0,53.0,0.0,0.0,0.0,0.0,Loss


Remove nulls

In [51]:
gameplay['target_catch_perc_RunningBacks'].fillna(0, inplace=True)
gameplay.dropna(axis=0, how='any', inplace=True)
gameplay.describe().T[:60]

,count,mean,std,min,25%,50%,75%,max
completed_passes_Quarterbacks,2549.0,22.197332,5.987157,1.00,18.00,22.00,26.00,45.00
attempted_passes_Quarterbacks,2549.0,34.808552,8.312872,7.00,29.00,34.00,40.00,68.00
passing_yards_Quarterbacks,2549.0,250.476658,76.026115,13.00,198.00,246.00,300.00,624.00
passing_touchdowns_Quarterbacks,2549.0,1.567674,1.178626,0.00,1.00,1.00,2.00,6.00
interceptions_thrown_Quarterbacks,2549.0,0.803845,0.945518,0.00,0.00,1.00,1.00,6.00
times_sacked_Quarterbacks,2549.0,2.333464,1.727122,0.00,1.00,2.00,3.00,11.00
yards_lost_from_sacks_Quarterbacks,2549.0,15.409965,12.649306,0.00,6.00,13.00,22.00,72.00
rush_attempts_Quarterbacks,2549.0,3.392703,2.833134,0.00,1.00,3.00,5.00,26.00
rush_yards_Quarterbacks,2549.0,14.752060,19.532842,-12.00,0.00,8.00,22.00,152.00
rush_touchdowns_Quarterbacks,2549.0,0.155747,0.401708,0.00,0.00,0.00,0.00,3.00


In [52]:
gameplay.describe().T[60:]

,count,mean,std,min,25%,50%,75%,max
field_goals_made_Kickers,2549.0,1.611612,1.280131,0.0,1.0,1.0,2.0,8.0
field_goals_attempted_Kickers,2549.0,1.921146,1.389740,0.0,1.0,2.0,3.0,9.0
punts_Kickers,2549.0,4.292664,1.893730,0.0,3.0,4.0,6.0,12.0
total_punt_yards_Kickers,2549.0,195.459788,89.504175,0.0,131.0,189.0,253.0,549.0
yards_per_punt_Kickers,2549.0,45.182307,7.145963,0.0,42.0,45.5,48.8,88.0
kickoff_returns,2549.0,1.941938,1.516188,0.0,1.0,2.0,3.0,8.0
kickoff_return_yards,2549.0,43.291879,37.793368,-1.0,16.0,37.0,64.0,236.0
kickoff_return_touchdown,2549.0,0.012946,0.113065,0.0,0.0,0.0,0.0,1.0
punt_returns,2549.0,1.811691,1.366740,0.0,1.0,2.0,3.0,8.0
punt_return_yards,2549.0,15.067870,18.970286,-15.0,0.0,10.0,21.0,195.0


Save dataframe

In [53]:
gameplay.to_csv('D:/Data/FullGameStats.csv', index=False)